In [14]:
import pandas as pd
from nltk.corpus import stopwords
import re

In [29]:
df = pd.read_csv('sochi.csv', index_col=0)

In [25]:
df.head()

,mark,review_text,date
0,1,\nТуалеты на набережной Сервис Сочи оставляет ...,14 апреля 2023
1,6,\nЕсли нет возможности отдохнуть в Турции Отды...,31 октября 2022
2,1,\nХозяин ГД Лиза - непорядочно поступает!!!! Г...,28 сентября 2022
3,2,"\nПри поездке запаситесь регидроном, полисорбо...",25 сентября 2022
4,1,\nМрак! Ужасный город. Постоянно отключают вод...,23 сентября 2022


In [54]:
def remove_stopwords(text: str) -> str:
    russian_stopwords = stopwords.words("russian")
    russian_stopwords = [
        word for word in russian_stopwords if word not in ("не", "нет", "да")
    ]
    tokens = [word for word in text.split(" ") if word not in russian_stopwords]
    text = " ".join(tokens)
    return text

def preprocess(x):
    
    x = x.lower()
    
    # Удаление html тегов
    x = re.sub("(<.*?>)", " ", x)
    x = re.sub('(«|»|")', " ", x)
    
    # ссылки
    x = re.sub(
        r"((https?):((//)|(\\\\))+([<?>?\w\d:#@%/;$()~_?\+-=\\\.&…](#!)?)*)",
        "",
        x,
    )
    
    x = re.sub("\s+", " ", x)
    
    x = remove_stopwords(x)
    
    x = re.sub("_", " ", x)
    
    return x
    

In [55]:
df.review_text = df.review_text.map(lambda x: preprocess(x))

In [57]:
import os

for file in os.listdir('data'):
    if os.path.isdir(f'data/{file}'): continue
    clean = pd.read_csv(f'data/{file}', index_col=0)
    clean.review_text = clean.review_text.map(lambda x: preprocess(x))
    clean.to_csv(f'data/{file[:file.find(".")]}_clean.csv')

PermissionError: [Errno 13] Permission denied: 'data/clean'

In [50]:
clean_files = os.listdir('data/clean')
final = pd.read_csv(f'data/clean/{clean_files[0]}', index_col=0)

for file in clean_files[1:]:
    temp = pd.read_csv(f'data/clean/{file}', index_col=0)
    final = pd.concat([final, temp], ignore_index=True)

In [53]:
final.to_csv('dataset.csv')